In [1]:
import findspark
findspark.init()

In [2]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pickle

from pyspark.sql import SparkSession
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from models.bloom_filter import BloomFilterBasedModel, BloomFilter
from models.emb_logreg import EmbeddingsLogReg

In [4]:
spark = SparkSession.builder \
    .appName("WikimediaStreamProcessor") \
    .config("spark.driver.memory", "12G") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.5.1") \
    .getOrCreate()

    # .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    # .config("spark.kryoserializer.buffer.max", "2000M") \

24/11/18 13:49:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Exception in thread "main" java.nio.file.NoSuchFileException: /tmp/tmpemsklmwi/connection6492578444515620286.info
	at sun.nio.fs.UnixException.translateToIOException(UnixException.java:86)
	at sun.nio.fs.UnixException.rethrowAsIOException(UnixException.java:102)
	at sun.nio.fs.UnixException.rethrowAsIOException(UnixException.java:107)
	at sun.nio.fs.UnixFileSystemProvider.newByteChannel(UnixFileSystemProvider.java:214)
	at java.nio.file.Files.newByteChannel(Files.java:361)
	at java.nio.file.Files.createFile(Files.java:632)
	at java.nio.file.TempFileHelper.create(TempFileHelper.java:138)
	at java.nio.file.TempFileHelper.createTempFile(TempFileHelper.java:161)
	at java.nio.file.Files.createTempFile(Files.java:852)
	at o

:: loading settings :: url = jar:file:/home/andrii/spark-3.5.3-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/andrii/.ivy2/cache
The jars for the packages stored in: /home/andrii/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f818c1b0-2300-47fd-90a4-ae608c3e72bf;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.5.1 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in centra

In [5]:
train_df = spark.read.csv(
    "./data/train/train_dataset.csv",
    header=True,
    inferSchema=True,
    multiLine=True,
    escape="\"",
    quote="\"",
    sep=",",
)

In [6]:
train_df_wtext = train_df.withColumn(
    "text",
    F.concat(
        F.col("title"),
        F.when(F.col("comment").isNotNull(), F.col("comment")).otherwise(F.lit("NULL"))
    )
)

In [7]:
train_df_wtext = train_df_wtext.withColumn("label", F.udf(lambda x: 1 if x else 0, T.IntegerType())(F.col("bot")))

In [8]:
total_rows = train_df_wtext.count()
train_split_index = int(total_rows * 0.7)

train_df_wtext = train_df_wtext.withColumn("row_index", F.monotonically_increasing_id())

train_df_ = train_df_wtext.filter(F.col("row_index") <= train_split_index).drop("row_index")
val_df = train_df_wtext.filter(F.col("row_index") > train_split_index).drop("row_index")

print("Training Dataset Count: " + str(train_df_.count()))
print("Val Dataset Count: " + str(val_df.count()))


Training Dataset Count: 3501
Val Dataset Count: 1499


In [ ]:
embeddingLogReg = EmbeddingsLogReg(10, 0.3, 0)

In [ ]:
embeddingLogReg.fit(train_df_)

In [ ]:
predictions = embeddingLogReg.predict(val_df)

In [ ]:
val_labels = predictions.select("label").rdd.flatMap(lambda x: x).collect()
val_preds = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

In [ ]:
accuracy_score(val_labels, val_preds), precision_score(val_labels, val_preds), recall_score(val_labels, val_preds), f1_score(val_labels, val_preds)

In [ ]:
embeddingLogReg.save("./data/logreg_small")

In [17]:
lr2 = EmbeddingsLogReg(0, 0, 0)

In [18]:
lr2 = EmbeddingsLogReg.load("./data/logreg_small")

In [19]:
predictions2 = lr2.predict(val_df)

In [20]:
val_labels_2 = predictions2.select("label").rdd.flatMap(lambda x: x).collect()
val_preds_2 = predictions2.select("prediction").rdd.flatMap(lambda x: x).collect()

In [21]:
accuracy_score(val_labels_2, val_preds_2), precision_score(val_labels_2, val_preds_2), recall_score(val_labels_2, val_preds_2), f1_score(val_labels_2, val_preds_2)

(0.9172781854569713,
 np.float64(0.9635036496350365),
 np.float64(0.8354430379746836),
 np.float64(0.8949152542372881))

In [9]:
bloom_filter = BloomFilterBasedModel(spark, fpr=0.1)

In [10]:
bloom_filter.fit(train_df_)

In [11]:
predictions = bloom_filter.predict(val_df)

In [12]:
val_labels = predictions.select("label").rdd.flatMap(lambda x: x).collect()
val_preds = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

accuracy_score(val_labels, val_preds), precision_score(val_labels, val_preds), recall_score(val_labels, val_preds), f1_score(val_labels, val_preds)

(0.933955970647098,
 np.float64(0.8645690834473324),
 np.float64(1.0),
 np.float64(0.9273661041819515))

In [ ]:
bloom_filter.save("./data/filter_train_small")

In [ ]:
filter2 = BloomFilterBasedModel.load(spark, "./data/filter_train_small")

In [ ]:
triggered_users = predictions.filter(F.col("prediction") == 1).select("user").distinct().rdd.map(lambda r: r[0]).collect()

In [ ]:
train_bots = train_df_.filter(F.col("bot")).select("user").distinct().rdd.map(lambda r: r[0]).collect()

In [ ]:
val_bots = val_df.filter(F.col("bot")).select("user").distinct().rdd.map(lambda r: r[0]).collect()

In [ ]:
set(train_bots) & set(val_bots)

{'AnomieBOT',
 'Cewbot',
 'Citation bot',
 'Community Tech bot',
 'DeltaQuadBot',
 'InternetArchiveBot',
 'JJMC89 bot III',
 'ListeriaBot',
 'RMCD bot',
 'SDZeroBot',
 'SineBot',
 'WoodwardBot'}

In [ ]:
set(val_bots) & set(triggered_users)

{'AnomieBOT',
 'Cewbot',
 'Citation bot',
 'Community Tech bot',
 'DeltaQuadBot',
 'FireflyBot',
 'InternetArchiveBot',
 'JJMC89 bot III',
 'ListeriaBot',
 'RMCD bot',
 'SDZeroBot',
 'SineBot',
 'WoodwardBot'}

In [ ]:
set(train_bots) & set(triggered_users)

{'AnomieBOT',
 'Cewbot',
 'Citation bot',
 'Community Tech bot',
 'DeltaQuadBot',
 'InternetArchiveBot',
 'JJMC89 bot III',
 'ListeriaBot',
 'RMCD bot',
 'SDZeroBot',
 'SineBot',
 'WoodwardBot'}

In [ ]:
filter2.bloom_filter.lookup('Georgymm')

True

In [ ]:
set(triggered_users) - set(train_bots)

{'103.83.219.61',
 '12.75.128.73',
 '141.156.183.94',
 '198.54.211.2',
 '2001:A61:124D:8201:3DB1:FC22:44A8:4133',
 '205.254.171.234',
 '219.89.42.235',
 '219.89.46.225',
 '2406:3003:2005:2A6B:D25:63D5:EC1E:EFBE',
 '2407:D000:F:78C4:64F6:5277:62C7:969D',
 '39.40.114.200',
 '98.149.97.245',
 'Acratopotes',
 'AideDésintéressée',
 'Alicyi',
 'Ardyjofry',
 'Argybz',
 'Baloug',
 'BlackMafoon12',
 'Ca',
 'Chiswick Chap',
 'CoconutOctopus',
 'Crunchydillpickle',
 'DankJae',
 'Dave-okanagan',
 'Drchriswilliams',
 'FireflyBot',
 'Georgymm',
 'Golem469',
 'Goswami21',
 'How15948',
 'IowaHistorian1997',
 'Ipigott',
 'MacronMonitor',
 'Marcocapelle',
 'Mashfi23',
 'Migsbasinillo',
 'Mill 1',
 'Mjks28',
 'MountVic127',
 'Nicknack009',
 'Nicolaj62',
 'Nimbus227',
 'Ninud',
 'Rif Winfield',
 'Supersaints2014',
 'Syahir20',
 'TempusTacet',
 'Tonypicc',
 'Unknown Temptation',
 'Vexald',
 'Viatori',
 'Ybot3000'}

In [ ]:
untriggered_users = predictions.filter((F.col("prediction") == 0) & (F.col("label") == 0)).select("user").distinct().rdd.map(lambda r: r[0]).collect()

In [ ]:
len(set(untriggered_users)), len(set(triggered_users))

(343, 65)

In [ ]:
predictions = filter2.predict(val_df)

In [ ]:
val_labels = predictions.select("label").rdd.flatMap(lambda x: x).collect()
val_preds = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

accuracy_score(val_labels, val_preds), precision_score(val_labels, val_preds), recall_score(val_labels, val_preds), f1_score(val_labels, val_preds)

(0.933955970647098,
 np.float64(0.8645690834473324),
 np.float64(1.0),
 np.float64(0.9273661041819515))

In [ ]:
spark.stop()